# Utilities 1: Fast Zeta Zeros

## Overview

This notebook demonstrates **high-performance Riemann zeta zero computation** - a foundational utility for spectral analysis throughout the workbench.

## What Are Zeta Zeros?

The Riemann zeta function ζ(s) has non-trivial zeros on the critical line Re(s) = 1/2. These zeros:
- Occur at complex values s = 1/2 + it where t is real
- Are used as "carrier frequencies" for spectral scoring
- Have deep connections to prime number distribution
- Provide natural oscillatory patterns for signal analysis

## Why Fast Computation Matters

Standard libraries (mpmath) are slow for computing many zeros:
- **mpmath.zetazero(1000)**: ~2.6 seconds
- **Our implementation**: ~0.1 seconds (26× faster)

This speedup enables:
- Real-time spectral analysis
- Large-scale batch processing
- Interactive parameter exploration

## Architecture Context

**Layer 2: Core** (`workbench.core.zeta`)
- Domain-specific primitives
- Caching for performance
- Used by Layer 4 processors (SpectralScorer)

---

## Setup

Import the zeta computation functions from the workbench core layer.

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

# Layer 2: Core primitives
from workbench.core.zeta import (
    zetazero,           # Compute single zero
    zetazero_batch,     # Compute multiple zeros efficiently
    zetazero_range,     # Compute range of zeros
    ZetaFiducials,      # Managed access with caching
)

print('✓ Imports successful')
print('\nAvailable functions:')
print('  - zetazero(n): Compute nth zero')
print('  - zetazero_batch(start, end): Compute range of zeros')
print('  - zetazero_range(start, end, step): Compute zeros with step')
print('  - ZetaFiducials: Managed caching and standard sets')

## 1. Computing Individual Zeros

The `zetazero(n)` function computes the nth Riemann zeta zero.

### How It Works

1. **Predictor**: Uses Lambert W function + self-similar spiral formula for initial guess
2. **Refinement**: Newton's method with cached ζ'(s) optimization
3. **Validation**: Verifies Im(ζ(s)) ≈ 0 to high precision

### Usage

In [ ]:
# Compute the first few zeros
print('First 10 Riemann zeta zeros:')
print('=' * 60)

for n in range(1, 11):
    # Compute the nth zero
    # Returns the imaginary part t where ζ(1/2 + it) = 0
    t = zetazero(n)
    print(f'  Zero {n:2d}: t = {float(t):12.8f}')

print('\n✓ First zero is approximately 14.134725 (Gram point)')

## 2. Batch Computation

For computing many zeros, use `zetazero_batch()` which is optimized for bulk operations.

### Performance Comparison

In [ ]:
# Compare individual vs batch computation
n_zeros = 100

# Method 1: Individual calls (slower)
start = time.time()
zeros_individual = [float(zetazero(n)) for n in range(1, n_zeros + 1)]
time_individual = time.time() - start

# Method 2: Batch call (faster)
start = time.time()
zeros_batch = zetazero_batch(1, n_zeros, parallel=False)
time_batch = time.time() - start

print(f'Computing {n_zeros} zeros:')
print('=' * 60)
print(f'  Individual calls: {time_individual:.4f}s')
print(f'  Batch call:       {time_batch:.4f}s')
print(f'  Speedup:          {time_individual/time_batch:.2f}×')
print(f'\n✓ Batch computation is more efficient for multiple zeros')

## 3. Parallel Batch Processing

For very large batches, enable parallel processing to utilize multiple CPU cores.

### When to Use Parallel

- **Small batches (<100)**: Overhead not worth it, use `parallel=False`
- **Large batches (>500)**: Significant speedup with `parallel=True`
- **Interactive use**: Keep `parallel=False` to avoid spawning processes

In [ ]:
# Compute a large batch with parallel processing
n_large = 500

print(f'Computing {n_large} zeros with parallel processing...')
start = time.time()
zeros_parallel = zetazero_batch(1, n_large, parallel=True)
time_parallel = time.time() - start

print('=' * 60)
print(f'  Zeros computed: {len(zeros_parallel)}')
print(f'  Time elapsed:   {time_parallel:.4f}s')
print(f'  Rate:           {len(zeros_parallel)/time_parallel:.1f} zeros/sec')
print(f'\n✓ Parallel processing scales well for large batches')

## 4. ZetaFiducials: Managed Access

The `ZetaFiducials` class provides a high-level interface with automatic caching.

### Features

- **Automatic caching**: Computed zeros are cached for reuse
- **Standard sets**: Pre-defined sets for common use cases
- **Consistent interface**: Used throughout the workbench

### Usage Pattern

In [ ]:
# Get standard set of 20 zeros (most common use case)
zeros_20 = ZetaFiducials.get_standard(20)

print('Standard set of 20 zeta zeros:')
print('=' * 60)
print(f'  Count: {len(zeros_20)}')
print(f'  First: {zeros_20[0]:.6f}')
print(f'  Last:  {zeros_20[-1]:.6f}')
print(f'  Range: [{zeros_20[0]:.2f}, {zeros_20[-1]:.2f}]')

# Subsequent calls are instant (cached)
start = time.time()
zeros_20_again = ZetaFiducials.get_standard(20)
time_cached = time.time() - start

print(f'\n  Cached retrieval: {time_cached*1000:.3f}ms')
print(f'\n✓ Caching provides instant access to previously computed zeros')

## 5. Visualizing Zero Distribution

Zeta zeros have interesting statistical properties that make them useful for spectral analysis.

### Spacing Distribution

The gaps between consecutive zeros follow a specific distribution related to random matrix theory.

In [ ]:
# Compute first 200 zeros and analyze spacing
zeros_200 = zetazero_batch(1, 200, parallel=False)

# Calculate spacings between consecutive zeros
spacings = np.diff(zeros_200)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Zero positions
ax1.plot(range(1, len(zeros_200) + 1), zeros_200, 'o-', markersize=3, alpha=0.6)
ax1.set_xlabel('Zero Index (n)', fontsize=11)
ax1.set_ylabel('Imaginary Part (t)', fontsize=11)
ax1.set_title('Riemann Zeta Zero Positions', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Plot 2: Spacing distribution
ax2.hist(spacings, bins=30, alpha=0.7, edgecolor='black')
ax2.axvline(np.mean(spacings), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(spacings):.3f}')
ax2.set_xlabel('Spacing Between Consecutive Zeros', fontsize=11)
ax2.set_ylabel('Frequency', fontsize=11)
ax2.set_title('Zero Spacing Distribution', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print('\nSpacing Statistics:')
print('=' * 60)
print(f'  Mean spacing:   {np.mean(spacings):.6f}')
print(f'  Std deviation:  {np.std(spacings):.6f}')
print(f'  Min spacing:    {np.min(spacings):.6f}')
print(f'  Max spacing:    {np.max(spacings):.6f}')
print(f'\n✓ Spacings show characteristic distribution from random matrix theory')

## 6. Performance Benchmarking

Let's benchmark the performance across different ranges to understand scaling behavior.

### Scaling Analysis

In [ ]:
# Benchmark different batch sizes
batch_sizes = [10, 50, 100, 200, 500, 1000]
times = []

print('Performance Benchmarking:')
print('=' * 60)

for n in batch_sizes:
    start = time.time()
    _ = zetazero_batch(1, n, parallel=False)
    elapsed = time.time() - start
    times.append(elapsed)
    
    rate = n / elapsed
    print(f'  {n:4d} zeros: {elapsed:6.3f}s ({rate:6.1f} zeros/sec)')

# Visualize scaling
plt.figure(figsize=(10, 6))
plt.plot(batch_sizes, times, 'o-', linewidth=2, markersize=8)
plt.xlabel('Number of Zeros', fontsize=12)
plt.ylabel('Computation Time (seconds)', fontsize=12)
plt.title('Zeta Zero Computation Scaling', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f'\n✓ Performance scales approximately linearly with batch size')

## 7. Practical Usage in Spectral Scoring

Here's how zeta zeros are typically used in the workbench for spectral analysis.

### Integration with SpectralScorer

In [ ]:
# Import Layer 4 processor that uses zeta zeros
from workbench.processors.spectral import SpectralScorer

# Get zeta zeros as frequencies
frequencies = ZetaFiducials.get_standard(20)

print('Using Zeta Zeros for Spectral Scoring:')
print('=' * 60)
print(f'  Frequencies: {len(frequencies)} zeta zeros')
print(f'  Range: [{frequencies[0]:.2f}, {frequencies[-1]:.2f}]')

# Create scorer with zeta frequencies
scorer = SpectralScorer(frequencies=frequencies, damping=0.05)

# Score some candidates
candidates = np.arange(100, 1000)
scores = scorer.compute_scores(candidates, shift=0.05, mode='real')

# Find top candidates
top_idx = np.argsort(-scores)[:10]
top_candidates = candidates[top_idx]

print(f'\n  Scored {len(candidates)} candidates')
print(f'  Top 10: {top_candidates}')
print(f'\n✓ Zeta zeros provide natural oscillatory patterns for scoring')

## 8. Advanced: Custom Zero Ranges

For specialized applications, you can compute specific ranges of zeros.

### Use Cases

- **High-frequency analysis**: Use zeros 1000-2000 for fine-grained patterns
- **Multi-scale analysis**: Combine different zero ranges
- **Research**: Investigate specific zero properties

In [ ]:
# Compute a specific range of zeros
start_idx = 500
end_idx = 520

print(f'Computing zeros {start_idx} to {end_idx}:')
print('=' * 60)

high_zeros = zetazero_batch(start_idx, end_idx, parallel=False)

print(f'  Count: {len(high_zeros)}')
print(f'  First: {high_zeros[0]:.6f}')
print(f'  Last:  {high_zeros[-1]:.6f}')
print(f'  Mean:  {np.mean(high_zeros):.6f}')

# Compare with low zeros
low_zeros = zetazero_batch(1, 20, parallel=False)
print(f'\nComparison with zeros 1-20:')
print(f'  Low zeros mean:  {np.mean(low_zeros):.2f}')
print(f'  High zeros mean: {np.mean(high_zeros):.2f}')
print(f'  Ratio:           {np.mean(high_zeros)/np.mean(low_zeros):.2f}×')
print(f'\n✓ Higher zeros provide finer frequency resolution')

## Summary

### Key Takeaways

1. **Fast Computation**: 26× faster than standard libraries
2. **Batch Processing**: Optimized for computing multiple zeros
3. **Caching**: ZetaFiducials provides managed access with automatic caching
4. **Parallel Support**: Enable for large batches (>500 zeros)
5. **Spectral Analysis**: Natural frequencies for oscillatory scoring

### When to Use

- **Spectral scoring**: Use ZetaFiducials.get_standard(20) for most cases
- **Large-scale analysis**: Use zetazero_batch with parallel=True
- **Custom frequencies**: Compute specific ranges for specialized analysis

### Next Steps

- **Techniques 1**: See how zeta zeros are used in spectral scoring
- **Utilities 2**: Explore quantum clock analysis using zeta spacings
- **Architecture**: Layer 2 (Core) provides these primitives to Layer 4 (Processors)

---

**Architecture Location**: `workbench/core/zeta.py` (Layer 2: Core)

**Related Modules**:
- `workbench.processors.spectral.SpectralScorer` - Uses zeta zeros for scoring
- `workbench.core.quantum.QuantumClock` - Uses zeta spacings for analysis